# Описание


x1 = "верхний левый угол"  
y1 = "верхний левый угол"  
x2 = "верхний правый угол"  
y2 = "верхний правый угол"  
x3 = "нижний правый угол"  
y3 = "нижний правый угол"  
x4 = "нижний левый угол"  
y4 = "нижний левый угол"  
elevation = "высота"  
inclination = "максимальный угол наклона"  
temp = "температура"  
pressure = "давление"  
humidity = "влажность"  
wind_speed = "скорость ветра"  
wind_gust = "порыв ветра"  
clouds = "облачность"  
weather = "тип погоды"  
bdod =   
cec =   
cfvo =   
clay =   
nitrogen =   
ocd =  
ocs =  
phh2o =  
sand =  
silt =  
soc =  
score =  

# Создание модели

In [28]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn

Пока что с высотой и углом наклона не работаем

In [27]:
train_df1 = pd.read_pickle("./sample_data/vineyard6.pickle")
train_df1 = pd.DataFrame.from_dict(train_df1.values())
print(train_df1) 

           x1         y1        x2         y2        x3         y3        x4  \
0    7.938728  44.651236  7.940978  44.651236  7.938728  44.653486  7.940978   
1    7.940978  44.651236  7.943228  44.651236  7.940978  44.653486  7.943228   
2    7.943228  44.651236  7.945478  44.651236  7.943228  44.653486  7.945478   
3    7.945478  44.651236  7.947728  44.651236  7.945478  44.653486  7.947728   
4    7.947728  44.651236  7.949978  44.651236  7.947728  44.653486  7.949978   
..        ...        ...       ...        ...       ...        ...       ...   
763  7.981478  44.581486  7.983728  44.581486  7.981478  44.583736  7.983728   
764  7.983728  44.581486  7.985978  44.581486  7.983728  44.583736  7.985978   
765  7.985978  44.581486  7.988228  44.581486  7.985978  44.583736  7.988228   
766  7.988228  44.581486  7.990478  44.581486  7.988228  44.583736  7.990478   
767  7.990478  44.581486  7.992728  44.581486  7.990478  44.583736  7.992728   

            y4  elevation   temp  ...  

,col_1,col_2
0,3,a
1,2,b
2,1,c
3,0,d
